### Преамбула к ДЗ-1

В рамках даной работы вам нужно будет поработать с hadoop, а именно:
- Сложить в хранилище hdfs файлы для выполнения задания
- Использовать его команды на практике
- Написать скрипты для MapReduce задачи и добиться их успешного выполнения

**Важно!**
Старайтесь выполнить все задания в 1 ноутбуке. От всех запущенных комманд и скриптов нужны логи успешного выполнения этих команд. Так что, пожалуйста, не очищаейте логи ячеек перед отправкой.

Также структура и порядок ячеек в ноутбуке должен быть выстроен таким образом, чтобы мы сделали **Ran All** по вашему ноутбуку, и все ячейки были выполнены (пути до вмонтированных папок и нужных файлов в них мы учтем сами)

### Заметки по работе с докером

- Если у вас ОС Windows, то для запуска сборки вполне поможет [docker desktop](https://www.docker.com/products/docker-desktop/) (должен открываться без впн), в связке с [git bash](https://gitforwindows.org/) вам станут доступны команды `docker compose` прямо из консоли гит баша, где вы сможете запустить контейнер на своем железе
- Запускать юпитер ноутбук из неймноды (свободный порт 8888 будет прокинут в конфиге, через который вы сможете обращаться к запущенному ноутбуку), его можно запустить через команду на неймноде `python3 -m notebook --ip 0.0.0.0 --port 8888 --allow_remote_access=true --no-browser --allow-root --NotebookApp.token=my_secret`, тогда вместо рандомного токена при заходе в ноутбук вам просто можно будет ввести `my_secret`
- В текущем компоуз файле будет указан шаблон вмонтированных (volume) директорий для винды, вам же стоит поменять их на пути вашей ОС. Нужные файлы для дз можете добавлять в вмонтированную папку для неймноды, чтоб их было видно успешно внутри докера, и дальше вы могли их закинуть в hdfs.
- Также в текущей ситуации доступности докер хаба мы проверили зеркало гугла, образ для этой сборки будет тянуться оттуда (указан в dockerfile)

**1. Смотрим на Hadoop Distributed File System**

В рамках этой части вам нужно будет обращаться к HDFS с помощью CLI, разместить файлы для следующих заданий в распределеннй файловой системе и выполнить несколько преобразований над ними.

Для работы файлы можно скачать по следующим ссылкам:
- Текст Шекспира на английском [ссылка](https://drive.google.com/file/d/19-ohhp5APaIgq5tjeyL9kC1vh0ur1Up9/view?usp=sharing)
- Логи посещения сайтов юзерами за некоторый промежуток времени [ссылка](https://drive.google.com/file/d/1WXyq5WVSWwJYXPuH4kyAJ5mrR3XgfO_H/view?usp=sharing)
- Данные по доходам больших компаний, их филиалов в разных странах [ссылка](https://drive.google.com/file/d/1CuUdGdrayqqO7OBYCRXwcJv64g6EhIzJ/view?usp=sharing)

Разместите их в нашем внутреннем файловом хранилище докер-контейнера с помощью HDFS CLI, для дальнейшего удобства под каждый файл стоит создать каталог с простым и понятным именем, разместить сами файлы в разных каталогах.

Набор комманд, которые вам могут в этом помочь, доступны [здесь](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html)

В ячейках ниже должен быть полный набор комманд ваших обращей к консоли

In [1]:
!hdfs dfs -mkdir /input

mkdir: `/input': File exists


In [2]:
!hdfs dfs -mkdir /input/shakespeare_text

mkdir: `/input/shakespeare_text': File exists


In [3]:
!hdfs dfs -mkdir /input/session_logs

mkdir: `/input/session_logs': File exists


In [4]:
!hdfs dfs -mkdir /input/company_gain

mkdir: `/input/company_gain': File exists


In [5]:
!hdfs dfs -put ./text.txt /input/shakespeare_text/text.txt

put: `/input/shakespeare_text/text.txt': File exists


In [6]:
!hdfs dfs -put ./logs.csv /input/session_logs/logs.csv

put: `/input/session_logs/logs.csv': File exists


In [7]:
!hdfs dfs -put ./gain.csv /input/company_gain/gain.csv

put: `/input/company_gain/gain.csv': File exists


In [8]:
!hdfs dfs -mkdir /output

mkdir: `/output': File exists


In [9]:
!hdfs dfs -mkdir /output/word_count_task

mkdir: `/output/word_count_task': File exists


In [10]:
!hdfs dfs -mkdir /output/max_income

mkdir: `/output/max_income': File exists


In [11]:
!hdfs dfs -mkdir /output/top_sites

mkdir: `/output/top_sites': File exists


In [12]:
!hdfs dfs -mkdir /tmp

mkdir: `/tmp': File exists


In [13]:
## вы можете обращаться к консоли из ноутбука таким способом
!hdfs dfs -ls /

Found 3 items
drwxr-xr-x   - root supergroup          0 2024-06-22 12:25 /input
drwxr-xr-x   - root supergroup          0 2024-06-23 17:33 /output
drwxr-xr-x   - root supergroup          0 2024-06-22 14:03 /tmp


In [14]:
%%bash
## или же использовать для этого меджик строчку в ячейке %%bash, как вам будет удобнее

hdfs dfs -ls /

Found 3 items
drwxr-xr-x   - root supergroup          0 2024-06-22 12:25 /input
drwxr-xr-x   - root supergroup          0 2024-06-23 17:33 /output
drwxr-xr-x   - root supergroup          0 2024-06-22 14:03 /tmp


In [15]:
!hdfs dfs -ls -R -h /input

drwxr-xr-x   - root supergroup          0 2024-06-22 12:27 /input/company_gain
-rw-r--r--   1 root supergroup     46.4 M 2024-06-22 12:27 /input/company_gain/gain.csv
drwxr-xr-x   - root supergroup          0 2024-06-22 12:27 /input/session_logs
-rw-r--r--   1 root supergroup     34.8 M 2024-06-22 12:27 /input/session_logs/logs.csv
drwxr-xr-x   - root supergroup          0 2024-06-22 12:26 /input/shakespeare_text
-rw-r--r--   1 root supergroup    144.8 K 2024-06-22 12:26 /input/shakespeare_text/text.txt


**2. Решаем задачи MapReduce**

**2.1 Подсчет слов в тексте**

В рамках данного задания вам нужно подсчитать кол-во слов в тексте Шекспира (файл приложен к ДЗ в архиве datasets), то есть необходимо реализовать базовый функционал утилиты word count.

**Важно** - подсчитывайте число только тех слов, длина которых больше 4 символов. Проводить процесс удаления знаков препинания и прочих символов **не нужно**

Ниже вам представлены ячейки, в которых вы должны описать структуру маппера/редьсюера и ниже вызвать их в bash-скрипте запуска MR-таски

In [16]:
%%writefile mapper.py
import sys
for l in sys.stdin:
    words = l.strip().split()
    for w in words:
        if len(w) > 4:
            print(w, 1, sep='+')


Overwriting mapper.py


In [17]:
%%writefile reducer.py
import sys

cur_w = None
word_cnt = 0

for l in sys.stdin:
    w, cnt = l.strip().split('+')
    cnt = int(cnt)
    
    if cur_w != w:
        if cur_w:
            print(cur_w, word_cnt, sep='\t')
        cur_w = w
        word_cnt = 0
    
    word_cnt += cnt

if cur_w:
    print(cur_w, word_cnt, sep='\t')


Overwriting reducer.py


В качестве проверки ваших python-скриптов до запуска MR таски можно произвести их запуск через консольные команды 

Тогда наша задача не будет выполняться через датаноды, а только на локальной машине, но в случае ошибок в скриптах вы увидите их и сможете исправить

In [33]:
! cat ./text.txt | python3 mapper.py | sort -k1,2 | python3 reducer.py

&amp;c.	1
&amp;c.]	2
'Ay.'	2
'Ay:'	2
'Heart's	4
'Hold,	1
'I'll	1
'Juliet.']	1
'Romeo	2
'Signior	1
'Then	1
'Twixt	1
'Tybalt	1
'When	1
'Where	1
'Wilt	1
'Yea,'	2
'Your	1
'banished,'	2
'dost	1
'fall'st	1
'gainst	1
'haviour	1
'music	2
'pothecary,	1
'silver	2
'tis;	1
'twas	3
'twere	1
'twill	1
'twixt	1
'twould	1
'villain'	1
'ware,	1
-What,	1
Abate	1
Above	1
Abraham	1
Abraham,	1
Abraham.	5
Accurs'd,	1
Accursed	1
Acquaint	1
Adding	1
Adieu,	1
Adversity's	1
Affection	1
Affliction	1
Afore	1
After	1
Against	1
Alack	2
Alack,	4
Alas,	4
Alike	1
Alive	1
Alone,	1
Amen!	1
Amen,	1
Among	2
Ancient	1
Angelica;	1
Anon,	2
Anon.	1
Anon?	1
Anselmo	1
Answer	1
Antony!	1
Apothecary.	5
Apothecary.]	1
Appear	1
April	1
Arise,	1
Arise;	1
Arms,	1
Ascend	1
Athwart	1
Attendants.	1
Attendants.]	2
Attendants;	1
Aurora's	1
Away,	1
Back,	1
Balthasar	1
Balthasar,	1
Balthasar.	12
Balthasar.]	4
Balthasar?	1
Banish'd	1
Bearing	1
Beautiful	1
Beauty	1
Because	3
Begot	1
Beguil'd,	1
Being	13
Believe	1
Belonging	1
Benedicite!	1
Benvo

Как только в данной проверке вы получите успешный и корректный результат, можете запустить Map Reduce в ячейке ниже

In [19]:
%%bash

hdfs dfs -rm -r /output/word_count_task/ || true

hadoop jar /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -D mapreduce.job.name="word-count" \
    -files ./mapper.py,./reducer.py \
    -mapper "python3 mapper.py" \
    -reducer "python3 reducer.py" \
    -input /input/shakespeare_text/text.txt\
    -output /output/word_count_task


Deleted /output/word_count_task
packageJobJar: [/tmp/hadoop-unjar8469513307670519106/] [] /tmp/streamjob4911404205386296338.jar tmpDir=null
2024-06-28 09:28:55 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.21.0.3:8032
2024-06-28 09:28:55 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.21.0.3:8032
2024-06-28 09:28:55 INFO  JobResourceUploader:907 - Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1719566703336_0001
2024-06-28 09:28:56 INFO  FileInputFormat:266 - Total input files to process : 1
2024-06-28 09:28:57 INFO  JobSubmitter:202 - number of splits:2
2024-06-28 09:28:57 INFO  JobSubmitter:298 - Submitting tokens for job: job_1719566703336_0001
2024-06-28 09:28:57 INFO  JobSubmitter:299 - Executing with tokens: []
2024-06-28 09:28:57 INFO  Configuration:2854 - resource-types.xml not found
2024-06-28 09:28:57 INFO  ResourceUtils:476 - Unable to find '

Мониторить процесс работы таски можно на nodemanager по порту 8088 (уже прокинут в конфиге), там будет UI, в котором будет видно вашу запущенную задачу и её статус.

Результат работы скрипта должен выглядеть следующим образом:

```bash
word count
abtr 6852
btoad 4237
stress 1932
zen 1885
```

In [20]:
%%bash
## запустите эту команду, чтобы вывести счетчик определенных слов, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер

hdfs dfs -cat /output/word_count_task/* | sort -t$'\t' -k2.2nr  | head -n 3

Romeo.	171
Juliet.	123
Servant.	19


**2.2 Поиск максимального дохода у филиалов компаний по разным странам**

В данном задании нужно поработать с  информацией о доходах больших компаний и их филиалах в разных странных за некоторый период. Данные представлены в след. формате: `страна;компания;дата;доход`. Вам необходимо найти максимальный доход у каждой компании в отдельности по каждой представленной стране в данных.

Результат работы скрипта должен выглядеть следующим образом:

```bash
country  company                    income
Austria  Rodriguez, Sims and Arias  123456
USA      Rios-Lewis                 789998
Canada   Thompson-Mendez            148345
```

**Рекомендации**

1. В рамках этой задачи вам могу пригодиться дополнительные параметры mr таски, отвечающие за настройку шаффла, и правил сортировки ключей внутри него. Почитать о примерах их использования можно [здесь](https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html#More_Usage_Examples).

2. Не рекомендуем использовать `\t` в качестве символа разделителя для сложного ключа (потому что по дефолту таб используется для разделения колонок данных, и ключом в таком случае будет только первая колонка до таба). Если вы будете собирать сложный ключ для нужной вам сортировки данных, лучше всего будет использовать другие симловы, к примеру `+, =`

3. Как и в задании 2.1, вы можете создать набор ячеек, куда занесете код своих мапперы и редьюсеры, и также соберете код запуска MR таски. Нам нужно, чтобы вы предоставили нам в качестве решения этой задачи все эти составляющие в рамках этого ноутбука.

In [79]:
%%writefile mapper_2.py
import sys
for l in sys.stdin:
    country, company, date, income = l.split(';')
    country=country.strip()
    company=company.strip()
    income = int(income)
    print(str(country + '.' + company), income, sep='+')

Overwriting mapper_2.py


In [80]:
%%writefile reducer_2.py
import sys

current_country_company = None
current_max_income = 0

for line in sys.stdin:
    country_company, income = line.split('+')
    country, company = country_company.split('.')
    income = int(income)
    
    if current_country_company != country_company:
        if current_country_company:
            country, company = current_country_company.split('.')
            print(f"{country}\t{company}\t{current_max_income}")
        current_country_company = country_company
        current_max_income = 0
    
    current_max_income = max(current_max_income, income)

if current_country_company:
    country, company = current_country_company.split('.')
    print(f"{country}\t{company}\t{current_max_income}")

Overwriting reducer_2.py


In [87]:
%%bash

hdfs dfs -rm -r /output/max_income/ || true

hadoop jar /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -D mapreduce.job.name="max_income" \
    -files ./mapper_2.py,./reducer_2.py \
    -mapper "python3 mapper_2.py" \
    -reducer "python3 reducer_2.py" \
    -input /input/company_gain/gain.csv \
    -output /output/max_income \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner
    

Deleted /output/max_income
packageJobJar: [/tmp/hadoop-unjar9054214301863675006/] [] /tmp/streamjob884789912868652158.jar tmpDir=null
2024-06-23 10:01:17 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.21.0.4:8032
2024-06-23 10:01:17 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.21.0.4:8032
2024-06-23 10:01:17 INFO  JobResourceUploader:907 - Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1719048969911_0006
2024-06-23 10:01:17 INFO  FileInputFormat:266 - Total input files to process : 1
2024-06-23 10:01:17 INFO  JobSubmitter:202 - number of splits:2
2024-06-23 10:01:18 INFO  JobSubmitter:298 - Submitting tokens for job: job_1719048969911_0006
2024-06-23 10:01:18 INFO  JobSubmitter:299 - Executing with tokens: []
2024-06-23 10:01:18 INFO  Configuration:2854 - resource-types.xml not found
2024-06-23 10:01:18 INFO  ResourceUtils:476 - Unable to find 'resour

In [21]:
%%bash
## запустите эту команду, чтобы вывести результат работы по определенным компаниям, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер
## укажите путь до той директории на hdfs, куда вы складывали результат

hdfs dfs -cat /output/max_income/* | \
grep -E "China.*?Rios-Lewis|India.*?Thompson-Mendez|Kazakhstan.*?Moore, Woods and Thomas|Madagascar.*?Chavez Group|New Zealand.*?Rodriguez, Sims and Arias" | \
column -t -s$'\t'

British Indian Ocean Territory (Chagos Archipelago)  Thompson-Mendez            974420
China                                                Rios-Lewis                 994876
India                                                Thompson-Mendez            953060
Kazakhstan                                           Moore, Woods and Thomas    994117
Madagascar                                           Chavez Group               986514
New Zealand                                          Rodriguez, Sims and Arias  888292


**2.3 Решаем задачу поиска самых посещаемых сайтов**

В данном задании нужно поработать с логом данных о посещении юзерами различных сайтов.
Формат данных: `url;временная метка`. Вам нужно вывести топ 5 сайтов по посещаемости в каждую из дат, которая представлена в наших данных.

Результат работы скрипта должен выглядеть следующим образом:

```bash
date        site                            count
2024-05-25  https://gonzales-bautista.com/  987
2024-05-25  https://smith.com/              654
2024-05-25  https://www.smith.com/          321
```

**Рекомендации**

1. Вам могу пригодиться дополнительные параметры mr таски, отвечающие за настройку шаффла, и правил сортировки ключей внутри него. Почитать о примерах их использования можно [здесь](https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html#More_Usage_Examples).

2. Не рекомендуем использовать `\t` в качестве символа разделителя для сложного ключа (потому что по дефолту таб используется для разделения колонок данных, и ключом в таком случае будет только первая колонка до таба). Если вы будете собирать сложный ключ для нужной вам сортировки данных, лучше всего будет использовать другие симловы, к примеру `+, =`.

3. Возможно, у вас не получится решить данную задачу за одну mr таску, тогда вы просто описываете в решении скрипты ваших мапперов, редьюсеров под каждую из mr тасок, которые вам нужно запустить для получения нужного результата.

**Важно** помнить, что любой маппер и редьюсер должен работать за O(1) памяти, и если вы будете создавать какой-то список, куда будете складывать какие-то данные, то он не должен быть размера O(n). Если такой момент в вашем решении будет, пожалуйста, поясните его текстово, что с вашими переменными все ок, и у них нет размера O(n). 

In [22]:
%%writefile mapper_3.py
import sys

for line in sys.stdin:
    line = line.strip()
    url, timestamp = line.split(';')
    date = timestamp.split()[0]
    print(f"{date}+{url}+1")

Overwriting mapper_3.py


In [52]:
%%writefile reducer_3.py

import sys
from heapq import nlargest

current_date = None
current_url = None
current_count = 0
top_sites = []

def print_top_sites(date, top_sites):
    for site, count in top_sites:
        print(f'{date}\t{site}\t{count}')

def get_top5_sites(sites):
    top_sites_sorted = nlargest(5, sites, key=lambda x: x[1])
    return top_sites_sorted

for line in sys.stdin:
    line = line.strip()
    date, url, count = line.split('+')
    count = int(count)

    if current_date == date:
        if current_url == url:
            current_count += count
        else:
            if current_url:
                top_sites.append((current_url, current_count))
                if len(top_sites) > 5:
                    top_sites = get_top5_sites(top_sites)
            current_url = url
            current_count = count
    else:
        if current_date:
            top_sites.append((current_url, current_count))
            if len(top_sites) > 5:
                top_sites = get_top5_sites(top_sites)
            print_top_sites(current_date, top_sites)
            top_sites = []

        current_date = date
        current_url = url
        current_count = count

if current_date == date:
    top_sites.append((current_url, current_count))
    if len(top_sites) > 5:
        top_sites = get_top5_sites(top_sites)
    print_top_sites(current_date, top_sites)


Overwriting reducer_3.py


In [53]:
%%bash

hdfs dfs -rm -r /output/top_sites/ || true

hadoop jar /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -D mapreduce.job.name="top_sites" \
    -files ./mapper_3.py,./reducer_3.py \
    -mapper "python3 mapper_3.py" \
    -reducer "python3 reducer_3.py" \
    -input /input/session_logs/logs.csv \
    -output /output/top_sites \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

Deleted /output/top_sites
packageJobJar: [/tmp/hadoop-unjar1554322140869427217/] [] /tmp/streamjob17647701335300896.jar tmpDir=null
2024-06-28 09:56:04 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.21.0.3:8032
2024-06-28 09:56:04 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.21.0.3:8032
2024-06-28 09:56:04 INFO  JobResourceUploader:907 - Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1719566703336_0006
2024-06-28 09:56:04 INFO  FileInputFormat:266 - Total input files to process : 1
2024-06-28 09:56:04 INFO  JobSubmitter:202 - number of splits:2
2024-06-28 09:56:04 INFO  JobSubmitter:298 - Submitting tokens for job: job_1719566703336_0006
2024-06-28 09:56:04 INFO  JobSubmitter:299 - Executing with tokens: []
2024-06-28 09:56:05 INFO  Configuration:2854 - resource-types.xml not found
2024-06-28 09:56:05 INFO  ResourceUtils:476 - Unable to find 'resource

In [55]:
%%bash

hdfs dfs -cat /output/top_sites/* | grep -E '2024-05-28|2024-06-02|2024-05-30' | column -t -s$'\t'

2024-05-28  https://gonzales-bautista.com/  368
2024-05-28  https://smith.com/              256
2024-05-28  https://www.smith.com/          251
2024-05-28  http://smith.com/               224
2024-05-28  http://www.smith.com/           204
2024-05-30  https://gonzales-bautista.com/  353
2024-05-30  https://smith.com/              246
2024-05-30  https://www.smith.com/          239
2024-05-30  http://smith.com/               229
2024-05-30  http://www.smith.com/           225
2024-06-02  http://smith.com/               7
2024-06-02  https://gonzales-bautista.com/  7
2024-06-02  https://www.williams.com/       6
2024-06-02  http://lee.com/                 5
2024-06-02  http://miller.com/              5
